In [1]:
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import datetime as dt
import time

In [ ]:
# portfolio of stocks
stocks = ['GOOGL', 'AMZN', 'AAPL', 'META', 'MSFT', 'NVDA', 'BABA', 'WMT', 'TSLA', 'DOCU', 'PANW', 'UNH', 'MSTR']

#### Download historical data from yf API

In [ ]:
import yfinance as yf

start_date = "2020-01-01"
end_date = datetime.now().strftime("%Y-%m-%d")

# Download historical data from yf API
data = yf.download(stocks, start=start_date, end=end_date, group_by='ticker')

# download data to csv
data.to_csv("stocks.csv")

[*********************100%***********************]  9 of 9 completed


#### Download analysis from csv downloaded data

In [4]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv("stocks.csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Forward fill missing values for each column
df = df.fillna(method='ffill')

# Convert all values to float
df = df.astype(float)

# rename tickers to more readable names, ^DJI -> DJ, ^GSPC -> SP, ^IXIC -> NQ
df.columns = pd.MultiIndex.from_tuples([
    (
        col[0].replace('^DJI', 'DJ').replace('^GSPC', 'SP').replace('^IXIC', 'NQ').replace('^GDAXI', 'DAX').replace('^N225', 'NKY').replace('RTY=F', 'RUT'),
        col[1]
    ) for col in df.columns
])


# Show the result
df.head()

AMZN                                                    TSLA  \
                 Open       High        Low      Close      Volume       Open   
Date                                                                            
2020-01-02  93.750000  94.900497  93.207497  94.900497  80580000.0  28.299999   
2020-01-03  93.224998  94.309998  93.224998  93.748497  75288000.0  29.366667   
2020-01-06  93.000000  95.184502  93.000000  95.143997  81236000.0  29.364668   
2020-01-07  95.224998  95.694504  94.601997  95.343002  80898000.0  30.760000   
2020-01-08  94.902000  95.550003  94.321999  94.598503  70160000.0  31.580000   

                                                          ...        WMT  \
                 High        Low      Close       Volume  ...       Open   
Date                                                      ...              
2020-01-02  28.713333  28.114000  28.684000  142981500.0  ...  36.483549   
2020-01-03  30.266666  29.128000  29.534000  266677500.0  ...  36.302454   
2020-01-06  30.104000  29.333332  30.102667  151995000.0  ...  36.035402   
2020-01-07  31.441999  30.224001  31.270666  268231500.0  ...  35.992442   
2020-01-08  33.232666  31.215334  32.809334  467164500.0  ...  35.697768   

                                                              AAPL             \
                 High        Low      Close      Volume       Open       High   
Date                                                                            
2020-01-02  36.799704  36.434438  36.508106  20294700.0  71.545897  72.598899   
2020-01-03  36.462068  36.093732  36.185814  16197600.0  71.765674  72.594063   
2020-01-06  36.247197  35.842027  36.112141  19336500.0  70.954181  72.444313   
2020-01-07  36.072248  35.667081  35.777580  20540700.0  72.415345  72.671348   
2020-01-08  35.829754  35.507463  35.654797  17627400.0  71.768079  73.526295   

                                               
                  Low      Close       Volume  
Date                                           
2020-01-02  71.292311  72.538521  135480400.0  
2020-01-03  71.608692  71.833298  146322800.0  
2020-01-06  70.703005  72.405670  118387200.0  
2020-01-07  71.845377  72.065155  108872000.0  
2020-01-08  71.768079  73.224403  132079200.0  

[5 rows x 45 columns]

In [5]:
# # Here you can plot the charts for every ticker

# # For every ticker, plot Close
# import matplotlib.pyplot as plt

# for ticker in df.columns.levels[0]:
#     plt.figure(figsize=(12, 6))
#     plt.plot(df[ticker]['Close'], label=ticker)
#     plt.title(f"{ticker} Close Price")
#     plt.xlabel("Date")
#     plt.ylabel("Close Price")
#     plt.legend()
#     plt.show()

#### Plot of Volume Profile for All Stocks

In [6]:
import plotly.io as pio
pio.renderers.default = 'vscode'
import plotly.express as px
import plotly.graph_objects as go

for stock in stocks:
    # Use the stock data from our df
    df_stock = df[stock][['Close', 'Volume']]

    # Convert the stock to datetime if needed
    if not isinstance(df_stock.index, pd.DatetimeIndex):
        df_stock.index = pd.to_datetime(df_stock.index)

    # plot
    hist = go.Histogram(
        y=df_stock['Close'],
        x=df_stock['Volume'],
        nbinsy=150,
        orientation='h',
        name='Volume Profile',
        marker_color='#B0C4DE',
        opacity=0.5,
        xaxis='x2',
    )

    line = go.Scatter(
        x=df_stock.index,
        y=df_stock['Close'],
        mode='lines',
        name='Close',
        marker_color='blue',
        xaxis='x',
        yaxis='y',
    )

    fig = go.Figure([hist, line])

    fig.update_layout(
        xaxis=dict(
            title='Date',
            anchor='y',
            domain=[0, 1],
            showgrid=False,
            side='bottom'
        ),
        xaxis2=dict(
            title='Sum of Volume',
            overlaying='x',
            side='top',
            showgrid=False
        ),
        yaxis=dict(
            title='Close Price',
            showgrid=True
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        ),
        height=600,
        title=f'Volume Profile & {stock} Close Prices'
    )

    fig.show()